# MSDS 7331 - Case Study 1 - Superconducting Materials
Daniel Crouthamel
Sophia Wu
Fabio Savorgnan
Bo Yun

## Business Understanding
--
You should always state the objective at the beginning of every case (a guideline you should follow in real life as well) and provide some initial "Business Understanding" statements (i.e., what is trying to be solved for and why might it be important)
--

**Objective:** 
The objective of this case study is to explore Linerar Regression with L1 and L2 regularization, and the impact to predicting the critical temperature of a superconductor. Additionally, feature importance is also investigated with the best model. Three models will be considered.

* Lasso (L1)
* Ridge (L2)
* Elastic Net (L1 and L2).

**Todo!!**
Summarize/background on L1/L2, the affect of alpha, l1 ratio, the hyperparameters.

Grid search will be performed on each model to find the optimal hyperparameters. We'll then use the optimal hyperparameters for each model and then determine which model performs the best. Along the way will explore the importance of normalization and scaling and condlude with a summary of the most important features.

## Data Evaluation and Engineering
--
Summarize the data being used in the case using appropriate mediums (charts, graphs, tables); address questions such as: Are there missing values? Which variables are needed (which ones are not)? What assumptions or conclusions are you drawing that need to be relayed to your audience?
--

Below we'll load our supoerconductor data and then perform a quick data exploration. Our data consists of two files which were merged together. Some initial obervations are:

* There is no missing data
* We have features with constant values
* There is one string object in the dataframe

In [1]:
import numpy as np
import pandas as pd

# Load Data
data_train = pd.read_csv('./data/train.csv')
data_materials = pd.read_csv('./data/unique_m.csv')

# Drop the duplicate column 'critical_temp' in the first frame
data_train = data_train.drop(['critical_temp'], axis=1)

# Merge the two frames
data = pd.merge(data_train, data_materials, left_index=True, right_index=True)

# Data frame to csv drop index
data.to_csv('./data/super_conducter_data.csv', index=False)

# Print out some typicaly descriptive statistics
print("")
data.info()
data.describe()

# Do we have missing data?
print("")
print("Missing Data?", data.columns[data.isnull().any().values])

# Columns with a Constant value
print("")
print("Columns that have the same value", data.columns[data.nunique() <= 1].values)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21263 entries, 0 to 21262
Columns: 169 entries, number_of_elements to material
dtypes: float64(156), int64(12), object(1)
memory usage: 27.4+ MB

Missing Data? Index([], dtype='object')

Columns that have the same value ['He' 'Ne' 'Ar' 'Kr' 'Xe' 'Pm' 'Po' 'At' 'Rn']


The string feature will be removed from our data set. This appears to be a name, and some values are duplicated. We felt it was save to remove. Addtionally, the features identified above with constant values will be removed.

In [2]:
# Drop columns with constant values
data.drop(columns=['material', 'He', 'Ne', 'Ar', 'Kr', 'Xe', 'Pm', 'Po', 'At', 'Rn'], inplace=True)
print(data.shape)

(21263, 159)


The code below is used to create a pandas profile, which is an easy way to see summary statistics for each feature. The html file will be provided as part of the case study submission. Some findings from that report are:

* Confirms no missing values
* There are no negative values
* Varying distrubutions for the features, some bell shaped, others poisson, etc.
* We have features with skewness and outliers.

To run the code, uncomment the last 3 lines of code below and run the cell. The first two lines indicate what needs to be installed. You will need to use an older version of pandas.

In [3]:
## install pandas 1.2.4
## pip install pandas-profiling==2.8.0

# from pandas_profiling import ProfileReport
# profile = ProfileReport(data, title="Pandas Profiling Report", minimal=True)
# profile.to_file(output_file="PandasProfile.html")

## Modeling Preparations
--
Which methods are you proposing to utilize to solve the problem?  Why is this method appropriate given the business objective? How will you determine if your approach is useful (or how will you differentiate which approach is more useful than another)?  More specifically, what evaluation metrics are most useful given that the problem is a regression one (ex., RMSE, logloss, MAE, etc.)?
--

As mentioned above, we'll be using grid search on three different models. For each model we'll use R2 as a cross validation metric, which can can explain how much of the variance is capatured by the model. We'll then use MAE (Mean Absolute Error) to differentiate between if Lasso, Ridge, or Elastic Net performs better.

## Model Building and Evaluation
--
In this case, your primary task is to build a linear regression model using L1 or L2 regularization (or both) to predict the critical temperature and will involve the following steps:

- Specify your sampling methodology
- Setup your model(s) - specifying the regularization type chosen and including the parameters utilized by the model
- Analyze your model's performance - referencing your chosen evaluation metric (including supplemental visuals and analysis where appropriate)
--

The code below creates 3 models and performs grid search on each model. For Lasso and Ridge, we'll vary the hyperparameter alpha. For Elastic Net, we'll vary the hyperparameters alpha and l1_ratio. Ranges are shown below in the code.

Normalization and scaling are performed on the data. There are several different ones that can be used, but in the end we decided to go with RobustScaler, which scales features using statistics that are robust to outliers.

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html

We additionally use a Pipeline to define the order of steps. By using a Pipeline with GridSearchCV we only scale the data in the training set. The test sets in each fold are then scaled using the same scaler.

https://scikit-learn.org/stable/modules/cross_validation.html
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
https://stats.stackexchange.com/questions/445259/combining-pca-feature-scaling-and-cross-validation-without-training-test-data


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline

X = data.drop(columns=['critical_temp']).copy(deep=True)
y = data.loc[:,'critical_temp'].copy(deep=True)

X_train, X_test, y_train, y_test =\
    train_test_split(X, y,
    test_size=0.2,
    random_state=1)

lasso_pipe_svc = make_pipeline(RobustScaler(), Lasso(random_state=1))
ridge_pipe_svc = make_pipeline(RobustScaler(), Ridge(random_state=1))
elastic_pipe_svc = make_pipeline(RobustScaler(), ElasticNet(random_state=1))

param_range = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 10, 100, 1000, 10000]
param_l1_ratio = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

param_grid_lasso = [{'lasso__alpha': param_range}]
param_grid_ridge = [{'ridge__alpha': param_range}]
param_grid_elastic = [{'elasticnet__alpha': param_range, 'elasticnet__l1_ratio': param_l1_ratio}]

gs_lasso = GridSearchCV(estimator=lasso_pipe_svc, param_grid=param_grid_lasso, scoring='r2', cv=5, n_jobs=-1)
gs_lasso.fit(X_train, y_train)

gs_ridge = GridSearchCV(estimator=ridge_pipe_svc, param_grid=param_grid_ridge, scoring='r2', cv=5, n_jobs=-1)
gs_ridge.fit(X_train, y_train)

gs_elastic = GridSearchCV(estimator=elastic_pipe_svc, param_grid=param_grid_elastic, scoring='r2', cv=5, n_jobs=-1)
gs_elastic.fit(X_train, y_train)

print("Lasso")
print(gs_lasso.best_score_)
print(gs_lasso.best_params_)
print("")

print("Ridge")
print(gs_ridge.best_score_)
print(gs_ridge.best_params_)
print("")

print("Elastic")
print(gs_elastic.best_score_)
print(gs_elastic.best_params_)
print("")

Lasso
0.7129520975572088
{'lasso__alpha': 0.3}

Ridge
0.7065312755243309
{'ridge__alpha': 1000}

Elastic
0.7083853891920866
{'elasticnet__alpha': 0.3, 'elasticnet__l1_ratio': 0.9}



Above we see that each model performs almost identically on the training data. Recall that L1 is used for Lasso, and L2 is used for Ridget. Elastic Net uses both. The optimal l1_ration hyperparameter found for ElasticNet is 0.9. It's moving into the direction of Lasso. If the ratio is 1, it's Lasso, if it's 0, it's Ridge.

Additionally, the optimal alpha found for Ridge is 1000, which is quite high. It's trying to penalize cofficients harder, which means we'll see more coefficients that are close to zero.

Next we'll perform predictions on our test data. Note that because we are using a pipeline, the scaler is only applied to the test data. This ensures that we have no leakage of information from the training data.
https://stackoverflow.com/questions/35388647/how-to-use-gridsearchcv-output-for-a-scikit-prediction

In [5]:
from sklearn import metrics

# Note the X_test gets run through the pipeline above! Very important, it means that the scaler is also run on the test data
y_lasso_pred = gs_lasso.predict(X_test)
y_ridge_pred = gs_ridge.predict(X_test)
y_elastic_pred = gs_elastic.predict(X_test)

print("Lasso")
print("R2 ->", metrics.r2_score(y_test, y_lasso_pred))
print("MAE ->", metrics.mean_absolute_error(y_test, y_lasso_pred))

print("Ridge")
print("R2 ->", metrics.r2_score(y_test, y_ridge_pred))
print("MAE ->", metrics.mean_absolute_error(y_test, y_ridge_pred))
print("")

print("Elastic")
print("R2 ->", metrics.r2_score(y_test, y_elastic_pred))
print("MAE ->", metrics.mean_absolute_error(y_test, y_elastic_pred))
print("")

Lasso
R2 -> 0.7197254814065868
MAE -> 13.660714491348584
Ridge
R2 -> 0.7264906957630274
MAE -> 13.359929361601818

Elastic
R2 -> 0.7146988481193569
MAE -> 13.811785860925973



Above we see that the R2 values are consistent with what we found on the training data. However, the Ridge model performans marginally better. In this case I'd be inclined to use the Lasso model because of the reduced number of features. Below we output the ABSOLUTE value of each coefficient, in sorted fashion. This helps to give an indication of which features are important. The sign doesn't matter.

In [6]:
lasso_weights = {data.columns[k]:abs(v) for k, v in enumerate(gs_lasso.best_estimator_['lasso'].coef_)}
dict(sorted(lasso_weights.items(), key=lambda item: item[1], reverse=True))

{'Ba': 12.747910288530564,
 'wtd_gmean_ThermalConductivity': 12.364521222651712,
 'wtd_mean_ThermalConductivity': 11.947603500510484,
 'range_atomic_mass': 6.955103730407503,
 'wtd_std_Valence': 6.560344059460298,
 'wtd_gmean_ElectronAffinity': 4.328328539587896,
 'Bi': 3.974908814567439,
 'wtd_entropy_atomic_mass': 3.9508095199980278,
 'wtd_entropy_ThermalConductivity': 3.924730652055885,
 'Ca': 3.3178161344471087,
 'wtd_entropy_FusionHeat': 2.806253200477188,
 'wtd_entropy_ElectronAffinity': 2.543573398784333,
 'mean_Density': 1.738656984740053,
 'Si': 1.6972961033719118,
 'mean_fie': 1.5836528543493753,
 'Sr': 1.3418330269172298,
 'wtd_range_atomic_mass': 1.2028148462719708,
 'range_atomic_radius': 1.1880683928528337,
 'As': 1.1522532857677548,
 'S': 1.0526391079045871,
 'wtd_std_FusionHeat': 1.0469974142475256,
 'mean_ThermalConductivity': 0.9839999275894172,
 'wtd_std_atomic_mass': 0.7363630703449185,
 'wtd_range_FusionHeat': 0.582011916925144,
 'wtd_range_Density': 0.557855764513

In [7]:
ridge_weights = {data.columns[k]:abs(v) for k, v in enumerate(gs_ridge.best_estimator_['ridge'].coef_)}
dict(sorted(ridge_weights.items(), key=lambda item: item[1], reverse=True))

{'Ba': 10.370112488683777,
 'wtd_mean_ThermalConductivity': 6.483779782189887,
 'wtd_std_Valence': 6.051373478910801,
 'wtd_gmean_ThermalConductivity': 5.803254805998027,
 'range_atomic_mass': 4.462011897968528,
 'Bi': 4.206544355861225,
 'wtd_std_ThermalConductivity': 3.8285152144058774,
 'wtd_gmean_ElectronAffinity': 3.589116895521526,
 'Ca': 3.410551989821297,
 'wtd_entropy_ElectronAffinity': 3.394887646825719,
 'wtd_std_atomic_mass': 3.196081179425849,
 'wtd_entropy_atomic_mass': 2.8317915478142077,
 'mean_ThermalConductivity': 2.7609890505042176,
 'wtd_entropy_FusionHeat': 2.5880223369522963,
 'gmean_ThermalConductivity': 2.524473443109448,
 'range_atomic_radius': 2.524093411262746,
 'Hg': 2.5220927025033046,
 'wtd_range_ElectronAffinity': 2.5219111686153455,
 'std_ElectronAffinity': 2.46421314705602,
 'Tl': 2.4546386358442813,
 'Ag': 2.302784507378022,
 'std_atomic_mass': 2.2744357552364254,
 'wtd_mean_atomic_radius': 2.237973420062889,
 'wtd_std_atomic_radius': 2.225423834408161

## Model Interpretability & Explainability
Using at least one of your models above (if multiple were trained):

- Which variable(s) was (were) ""most important"" and why?  How did you come to the conclusion and how should your audience interpret this?

In [8]:
data
col= data[['Ba','wtd_gmean_ThermalConductivity','wtd_mean_ThermalConductivity' ]]
col

,Ba,wtd_gmean_ThermalConductivity,wtd_mean_ThermalConductivity
0,0.20,0.621979,61.015189
1,0.10,0.619735,61.372331
2,0.10,0.619095,60.943760
3,0.15,0.620535,60.979474
4,0.30,0.624878,61.086617
...,...,...,...
21258,0.00,95.001493,111.537778
21259,2.00,1.577047,108.680590
21260,0.00,57.038314,57.400000
21261,0.00,58.781651,59.270000


In [9]:
#calculate interquartile range for Ba
q3, q1 = np.percentile(data['Ba'], [75 ,25])
iqr_BA = q3 - q1
#display interquartile range 
iqr_BA

1.35

In [11]:
#calculate interquartile range for Ba
q3, q1 = np.percentile(data['wtd_gmean_ThermalConductivity'], [75 ,25])
iqr_Ther_gmean = q3 - q1
#display interquartile range 
iqr_Ther_gmean

46.220757864326025

In [12]:
#calculate interquartile range for Ba
q3, q1 = np.percentile(data['wtd_mean_ThermalConductivity'], [75 ,25])
iqr_Ther_mean = q3 - q1
#display interquartile range 
iqr_Ther_mean

44.881957804581404

We decided to use the Lasso model because the performance as explained above. The most important variable in the lasso model were the following:

'Ba': 12.747910288530564,

'wtd_gmean_ThermalConductivity': 12.364521222651712,

'wtd_mean_ThermalConductivity': 11.947603500510484

As you can see these 3 variables are the only variables among the 159 variables choosen to run the model that have a weight above 10. Therefore, the 3 mentioned variables are the most important variable to predict critical temperature by the lasso model.

Because we had the necessity to normalize our data due to different distribution of values from the variables to provide appropriate variables to compare and work for the model, we interpret the variables accounting for our normalized data. We used the robust scalar for normalization. This Scaler removes the median and scales the data according to the quantile range (defaults to IQR: Interquartile Range). The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile).
Therefore, the interpretation of the most important 3 variables mentioned above are the following:
BA= for any increase in 1.35 of the BA variables we will have an increase in the critical temperature of 12.75, when we keep constant the othe variables.
Wtd_gmean_ThermalConductivity= for any increase in 46.22 of the wtd_gmean_ThermalConductivity variable we will have an increase in the critical temperature of 12.36, hen we keep constant the othe variables.
Wtd_mean_ThermalConductivity= for any increase in 44.89 of the wtd_mean_ThermalConductivity variable we will have an increase in the critical temperature of 11.95, hen we keep constant the othe variables.

The lasso model has MAE= 13.6; the general interpretation of this model for the user is that any prediction with this model will have a mean error of plus/minus 13.6.


## Case Conclusions
After all of your technical analysis and modeling; what are you proposing to your audience and why?  How should they view your results and what should they consider when moving forward?  Are there other approaches you'd recommend exploring?  This is where you "bring it all home" in language they understand.

We will recommend to our audience to keep in mind that the most important variables to relate to critical high temperature are Ba wtd_gmean_ThermalConductivity, wtd_mean_ThermalConductivity. Because those variables turned out to have the highest weight in the lasso regularization model. So, moving forward the audience should concentrate in these 3 variables in the control or manipulation of critical high temperature. Furthermore, it is very reassurance that the same variables came out in the other models that we run as the variables with higher weight of importance. For the future, it can be done a prospective study looking the relation with of these 3 variables and critical high temperature. 